In [1]:
import numpy as np
import pandas as pd
import gdal
from gdalconst import * 

import sys, os, time

from matplotlib import pyplot as plt
%matplotlib inline

from PIL import Image

from IPython.display import clear_output

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Load data in GeoTiff format

Let's look at the VIIRS data of night lights. This data is acquired monthly and is available since 2012. The spatial resolution is $0.5 \ km^2/px$.
The data is provided as a set of 6 tiles that span most of the globe (two files per tile).

In [ ]:
# list of files 

# files = """http://mapserver.ngdc.noaa.gov/viirs_data/viirs_composite/v10//201609/vcmcfg/SVDNB_npp_20160901-20160930_75N180W_vcmcfg_v10_c201610280941.tgz
# http://mapserver.ngdc.noaa.gov/viirs_data/viirs_composite/v10//201609/vcmcfg/SVDNB_npp_20160901-20160930_75N060W_vcmcfg_v10_c201610280941.tgz
# http://mapserver.ngdc.noaa.gov/viirs_data/viirs_composite/v10//201609/vcmcfg/SVDNB_npp_20160901-20160930_75N060E_vcmcfg_v10_c201610280941.tgz
# http://mapserver.ngdc.noaa.gov/viirs_data/viirs_composite/v10//201609/vcmcfg/SVDNB_npp_20160901-20160930_00N180W_vcmcfg_v10_c201610280941.tgz
# http://mapserver.ngdc.noaa.gov/viirs_data/viirs_composite/v10//201609/vcmcfg/SVDNB_npp_20160901-20160930_00N060W_vcmcfg_v10_c201610280941.tgz
# http://mapserver.ngdc.noaa.gov/viirs_data/viirs_composite/v10//201609/vcmcfg/SVDNB_npp_20160901-20160930_00N060E_vcmcfg_v10_c201610280941.tgz""".split("\n")

In [ ]:
rast_file = "/home/ubuntu/data/VIIRS/2016-09/SVDNB_npp_20160901-20160930_75N180W_vcmcfg_v10_c201610280941.avg_rade9.tif"

### Using plain GDAL

In [ ]:
rast = gdal.Open(rast_file, GA_ReadOnly)
rast

In [ ]:
cols = rast.RasterXSize
rows = rast.RasterYSize
bands = rast.RasterCount
driver = rast.GetDriver().LongName
print "Data size: %d x %d x %d (cols x rows x bands)"%(cols, rows, bands)
print driver

In [ ]:
geotransform = rast.GetGeoTransform()
originX = geotransform[0]
originY = geotransform[3]
pixelWidth = geotransform[1]
pixelHeight = geotransform[5]

In [ ]:
print originX, originY, pixelWidth, pixelHeight

In [ ]:
band = rast.GetRasterBand(1)
bandtype = gdal.GetDataTypeName(band.DataType)
print bandtype

In [ ]:
band.XSize, band.YSize

In [ ]:
data = band.ReadAsArray(0, 0, cols, rows)
value = data[3500,4000]
print value

data = band.ReadAsArray(0, 0, rast.RasterXSize, rast.RasterYSize).astype(np.float)
value = data[3500,4000]
print value


### Using rasterio

In [ ]:
import rasterio
rast2 = rasterio.open(rast_file)

In [ ]:
rast2.name, rast2.mode

In [ ]:
arr = rast2.read(1)

In [ ]:
arr.shape

In [ ]:
plt.imshow(arr[6000:8000,6000:8000], cmap='pink')
plt.show()  

# Extract regions of interest

### Utilities in satimg

In [ ]:
import sys
sys.path.append("../satellite-image-processing/")
import satimg 

In [ ]:
bounds = satimg.get_geotiff_bounds(rast)
bounds 

In [ ]:
loc = (37.7749, -122.4194) # San Francisco lat/lon

In [ ]:
bbox = satimg.bounding_box_at_location(loc, (100,100))
print bbox

In [ ]:
geoSize = satimg.km_to_deg_at_location(loc, (100,100))
print geoSize

In [ ]:
from pyproj import Proj, transform

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
x1,y1 = -11705274.6374,4826473.6922
x2,y2 = transform(inProj,outProj,x1,y1)
print x2,y2

### Using SatSense class

In [ ]:
simg = satimg.SatImage(rast_file)

In [ ]:
img = simg.get_image_at_location(loc, w=100)
img[0].shape

In [ ]:
plt.figure(figsize=(8,6))
plt.imshow(img[0], cmap='pink')

In [ ]:
import geocoder
locs = ["San Francisco, CA", "Cambridge, MA", "Toronto, Canada"]
locs = [tuple(geocoder.google(l).latlng) for l in locs]

In [ ]:
locs

In [ ]:
images = simg.get_image_at_locations(locs, w=100)

In [ ]:
for i in images.values():
    plt.figure(figsize=(8,4))
    plt.imshow(i[0], cmap='pink')

In [ ]:
satimg.generate_locations_within_bounding_box(bbox, 10)

In [ ]:
bbox

In [ ]:
from shapely.geometry import Polygon, Point
p = Polygon([(bbox[1], bbox[0]), ([bbox[1], bbox[2]]), (([bbox[3], bbox[2]])), ([bbox[3], bbox[0]])])

In [ ]:
satimg.generate_locations_within_polygon(p, 10)

In [ ]:
sampleImages = simg.sample_images_around_location(loc, w=5, W=100, nSamples=10)

In [ ]:
len(sampleImages)

In [ ]:
for l,i in sampleImages.iteritems():
    plt.imshow(i[0])
    plt.show()

### Open raster data with rasterio

In [ ]:
import rasterio
from rasterio.tools.mask import mask

# the polygon GeoJSON geometry
geoms = [{'type': 'Polygon', 
          'coordinates': [[(bbox[1], bbox[0]), ([bbox[1], bbox[2]]), (([bbox[3], bbox[2]])), ([bbox[3], bbox[0]])]]}]

# load the raster, mask it by the polygon and crop it
src = rasterio.open(rast_file)
out_image, out_transform = mask(src, geoms, crop=True)
out_meta = src.meta.copy()

print out_image.data.sum()

# # save the resulting raster  
# out_meta.update({"driver": "GTiff",
#     "height": out_image.shape[1],
#     "width": out_image.shape[2],
# "transform": out_transform})

# with rasterio.open("masked.tif", "w", **out_meta) as dest:
#     dest.write(out_image)

In [ ]:
plt.imshow(out_image.data[0,:,:])

# Display map using ipyleaflet

In [ ]:
# json
import json

# widgets
import ipyleaflet as ll

In [ ]:
m = ll.Map(center=loc, zoom=4)

In [ ]:
m

In [ ]:
with open('./europe_110.geo.json') as f:
    data = json.load(f)

for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }

layer = ll.GeoJSON(data=data, hover_style={'fillColor': 'red'})

def hover_handler(event=None, id=None, properties=None):
    sys.stdout.write("\r" + properties['geounit'])
    sys.stdout.flush()
    
layer.on_hover(hover_handler)
    
map.add_layer(layer)

map